## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
gis = GIS("home")

#Setup Google Earth Engine and ArcGIS Environment

#### Now you are ready to start!

In [2]:
pip install earthengine-api geopandas matplotlib arcgis


DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/tflite_model_maker-0.3.4-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.0/455.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 76.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.4/156.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1

#Authenticate and Initialize GEE in Python

In [9]:
import ee
import geemap
ee.Initialize()

# Define the region of interest (example: Kenya)
roi = ee.Geometry.Rectangle([33, -5, 42, 5])  # Bounding box for Kenya


In [4]:
pip install geemap


DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/tflite_model_maker-0.3.4-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 86.8 MB/s eta 0:00:00

In [8]:
import ee
ee.Authenticate()
ee.Initialize()


# Retrieve and Process Satellite Data

In [12]:
# Load Sentinel-2 data
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(roi).filterDate('2023-01-01', '2023-12-31')



# Calculate NDVI (Normalized Difference Vegetation Index)
def calc_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

ndvi_collection = sentinel2.map(calc_ndvi)

# Filter the image collection by cloud cover
ndvi_filtered = ndvi_collection.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))


#  Compute Standard Precipitation Index (SPI)

In [15]:
# Visualization parameters for SPI
spi_viz_params = {
    'min': -2.0,
    'max': 2.0,
    'palette': ['red', 'yellow', 'green'],
    'dimensions': 512,  # Add dimensions for pixel size
    'region': roi  # Specify the region of interest
}

# Generate the thumbnail URL for the SPI image with the region defined
spi_url = spi_image.getThumbURL(spi_viz_params)

# Create a folium map centered on the ROI
map_center = [0.0, 37.5]  # Center map on Kenya
m = folium.Map(location=map_center, zoom_start=6)

# Add SPI image to the folium map using ImageOverlay
folium.raster_layers.ImageOverlay(
    image=spi_url,
    bounds=[[roi.bounds().getInfo()['coordinates'][0][0][1], roi.bounds().getInfo()['coordinates'][0][0][0]],
            [roi.bounds().getInfo()['coordinates'][0][2][1], roi.bounds().getInfo()['coordinates'][0][2][0]]],
    opacity=0.6
).add_to(m)

# Display the map
m


# Map Water Resources

In [18]:
# Visualization parameters for water bodies
water_viz_params = {
    'min': 0,
    'max': 100,
    'palette': ['blue'],
    'dimensions': 2000,  # Set a limit on the output image dimensions to avoid exceeding limits
    'region': roi  # Clip to the region of interest
}

# Generate the thumbnail URL for the water bodies image with reduced dimensions
water_bodies_url = water_bodies.getThumbURL(water_viz_params)

# Create a folium map centered on the ROI
map_center = [0.0, 37.5]  # Center map on Kenya or your region
m = folium.Map(location=map_center, zoom_start=6)

# Add water bodies to the folium map using ImageOverlay
folium.raster_layers.ImageOverlay(
    image=water_bodies_url,
    bounds=[[roi.bounds().getInfo()['coordinates'][0][0][1], roi.bounds().getInfo()['coordinates'][0][0][0]],
            [roi.bounds().getInfo()['coordinates'][0][2][1], roi.bounds().getInfo()['coordinates'][0][2][0]]],
    opacity=0.6
).add_to(m)

# Display the map
m


# Visualization in ArcGIS using Python

In [31]:
from arcgis.gis import GIS
from arcgis.mapping import WebMap

# Authenticate with ArcGIS Online
gis = GIS("home")

# Search for available NDVI layers (inside or outside your organization)
search_results = gis.content.search('NDVI', 'Imagery Layer', max_items=10, outside_org=True)

# Check if any items were found before proceeding
if len(search_results) > 0:
    # Use the first search result for NDVI, adjust index if necessary
    item = search_results[0]  # Change this index to choose a different layer if needed

    # Access the first layer in the item (typically imagery layers have multiple layers)
    layer = item.layers[0]

    # Create a new WebMap object
    web_map = WebMap()

    # Set a predefined valid basemap (e.g., 'streets', 'satellite', 'dark-gray')
    web_map.basemap = 'streets'

    # Add the selected NDVI layer to the web map
    web_map.add_layer(layer)

    # Save and publish the web map with the required fields
    web_map_item = web_map.save({
        "title": "NDVI Map with USA NAIP Imagery",
        "tags": "NDVI, Vegetation, Drought",
        "snippet": "This map shows NDVI data from the USA NAIP Imagery layer."
    })

    # Output the entire item for inspection
    print("Web map item:", web_map_item)

    # Manually retrieve and print the URL if the item exists
    if web_map_item:
        print("Web map URL:", web_map_item.homepage)
    else:
        print("Web map was not saved correctly.")
else:
    print("No NDVI layers found. Please modify the search query.")


Web map item: <Item title:"NDVI Map with USA NAIP Imagery" type:Web Map owner:pinjarla.p_nu>
Web map URL: https://www.arcgis.com/home/item.html?id=05f7e1346bf14461957359603555b7b1
